## Урок 2
### Задание:
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии
- Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
- Ссылку на саму вакансию        
- Сайт откуда собрана вакансия

In [378]:
# Подключение нужных библиотек
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from pprint import pprint
import re

In [379]:
# Поиск по названию работы
vacancy = input('Работа: ')
city = input('Город: ')

Работа: грузчик
Город: krasnodar


In [363]:
def parser_hh(vacancy, city):
    # параметры поиска
    params = {
        'text': vacancy, \
    }
    
    '''
    Здесь код для page
    '''
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'
    }
    main_link_hh = f'https://{city}.hh.ru/search/vacancy?'
    
    response_hh = requests.get(main_link_hh, params=params, headers=headers).text
    soup = bs(response_hh,'lxml')
    
    # находим блоки вакансий
    vacancy_page = soup.find('div',{'class':'vacancy-serp'})
    
    # вытаскиваем списки вакансий 
    vacancy_list = vacancy_page.find_all('div',{'class':'vacancy-serp-item'})
    
    
    # словарь для содержания вакансий
    vacancys = []
    for job in vacancy_list:
        job_data = {}
        
        # название ваканасии 
        job_name = job.find('div',{'vacancy-serp-item__info'}).getText()
        job_data['name'] = job_name
        
        # ссылка на вакансию
        job_url = job.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        job_data['url'] = job_url
        
        # зарплата
        job_salary = job.find('div',{'class':'vacancy-serp-item__sidebar'}).getText()
        
        # признаюсь, поиск зарплаты не мой (https://github.com/dmitryrubtsov/Methods-of-collecting-and-processing-data-from-the-Internet/blob/master/lesson_02/lesson_02.ipynb)
        if not job_salary:
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            job_salary = job_salary.replace(u'\xa0', u'')
            job_salary = re.split(r'\s|-', job_salary)
            
            if job_salary[0] == 'до':
                salary_min = None
                salary_max = int(job_salary[1])
            elif job_salary[0] == 'от':
                salary_min = int(job_salary[1])
                salary_max = None
            else:
                salary_min = int(job_salary[0])
                salary_max = int(job_salary[1])
                
            salary_currency = job_salary[2]
            job_data['salary_min'] = salary_min
            job_data['salary_max'] = salary_max
            job_data['salary_currency'] = salary_currency
            
        
        # сайт вакансии
        job_data['main_link'] = main_link_hh
        
        # добавление данных в датасет
        vacancys.append(job_data)



In [785]:
def parser_sj(vacancy, city):
    params = {
        'keywords': vacancy
    }
    
    '''
    Здесь код для page
    '''
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'
    }
    main_link_sj = f'https://krasnodar.superjob.ru/vacancy/search/?'
    

    response_sj = requests.get(f'{main_link_sj}keywords={vacancy}', headers=headers).text
    soup = bs(response_sj,'lxml')
    
    # страница данных работы
    vacancy_page = soup.find('div',{'class':'_1Ttd8 _2CsQi'})
    # блок листы с работой
    vacancy_list = vacancy_page.find_all('div',{'class':'_3zucV _1fMKr undefined _1NAsu'})
    #pprint(job_list)
    
    vacancys = []
    for job in vacancy_list:
        job_data = {}
        
        # name
        salary_min = None
        salary_max = None
        salary_currency = None
        job_name = job.find('a',{'class':'_6AfZ9'})
        job_url = job.find('a',{'class':'_6AfZ9'})
        
        job_salary = job.find('span', {'class': 'f-test-text-company-item-salary'})
        if job.find('a',{'class':'_6AfZ9'}) == None:
            pass
        else:
            job_name = job.find('a',{'class':'_6AfZ9'}).getText()
            job_url = 'https://krasnodar.superjob.ru' + job.find('a',{'class':'_6AfZ9'})['href']
            # salary
            job_salary = job.find('span', {'class': '_1OuF_ _1qw9T f-test-text-company-item-salary'}).getText()
            job_salary = job_salary.replace(u'\xa0',u'')
            
            job_salarys = {}
            job_salary_check = re.findall(r'до|от', job_salary)
            job_salary_nums = re.findall(r'\d+', job_salary)
            
            job_salarys['check'] = job_salary_check
            job_salarys['nums'] = job_salary_nums
            
            
        if job_salary is None:
            pass
        elif job_salarys['check'] == 'до':
            salary_min = None
            salary_max = job_salarys['nums']
        elif job_salarys['check'] == 'от':
            salary_max = None
            salary_min = job_salarys['nums']
        else:
                salary_min = job_salarys['nums'][0]
                salary_max = job_salarys['nums'][-1]
        
        job_data['salary_max'] = salary_max
        job_data['salary_min'] = salary_min
        job_data['url'] = job_url
        job_data['name'] = job_name
        job_data['salary'] = job_salary
        
        vacancys.append(job_data)

In [795]:
parser_hh(vacancy, city)
#parser_sj(vacancy, city)

[{'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': 55000,
  'url': 'https://krasnodar.hh.ru/vacancy/32193668?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA'},
 {'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик-комплектовщик',
  'salary_currency': 'бел.руб.',
  'salary_max': None,
  'salary_min': 900,
  'url': 'https://krasnodar.hh.ru/vacancy/37480036?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA'},
 {'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик/разнорабочий',
  'salary_currency': 'руб.',
  'salary_max': 58300,
  'salary_min': 36800,
  'url': 'https://krasnodar.hh.ru/vacancy/37696324?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA'},
 {'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик/комплектовщик',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': 30000,
  'url': 'https://krasnodar.hh.ru/v

In [794]:
vacancy_date = []
vacancy_date.extend(parser_hh(vacancy,city))
vacancy_date.extend(parser_sj(vacancy, city))
    
df = pd.DataFrame(vacancy_date)

[{'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': 55000,
  'url': 'https://krasnodar.hh.ru/vacancy/32193668?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA'},
 {'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик-комплектовщик',
  'salary_currency': 'бел.руб.',
  'salary_max': None,
  'salary_min': 900,
  'url': 'https://krasnodar.hh.ru/vacancy/37480036?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA'},
 {'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик/разнорабочий',
  'salary_currency': 'руб.',
  'salary_max': 58300,
  'salary_min': 36800,
  'url': 'https://krasnodar.hh.ru/vacancy/37696324?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA'},
 {'main_link': 'https://krasnodar.hh.ru/search/vacancy?',
  'name': 'Грузчик/комплектовщик',
  'salary_currency': 'руб.',
  'salary_max': None,
  'salary_min': 30000,
  'url': 'https://krasnodar.hh.ru/v

TypeError: 'NoneType' object is not iterable

In [ ]:
df[833:853]